## Running SQL query in Jupyter notebook using %sql (or %%sql) magic  
(requires ipython-sql)

In [1]:
%load_ext sql

### Connect to database rating.db, using SQLAlchemy connect strings

In [3]:
%sql sqlite:///rating.db

u'Connected: None@rating.db'

In [11]:
%sql select * from sqlite_master

Done.


type,name,tbl_name,rootpage,sql
table,Movie,Movie,2,"CREATE TABLE Movie(mID int, title text, year int, director text)"
table,Reviewer,Reviewer,3,"CREATE TABLE Reviewer(rID int, name text)"
table,Rating,Rating,4,"CREATE TABLE Rating(rID int, mID int, stars int, ratingDate date)"


### Check table header

In [23]:
%sql select * from Movie LIMIT 1

Done.


mID,title,year,director
101,Gone with the Wind,1939,Victor Fleming


In [24]:
%sql select * from Rating LIMIT 1

Done.


rID,mID,stars,ratingDate
201,101,2,2011-01-22


In [25]:
%sql select * from Reviewer LIMIT 1

Done.


rID,name
201,Sarah Martinez


### Movie rating queries

Find the titles of all movies directed by Steven Spielberg.

In [12]:
%sql select title from Movie where director = "Steven Spielberg"

Done.


title
E.T.
Raiders of the Lost Ark


Find all years that have a movie that received a rating of 4 or 5, and sort them in increasing order.

In [13]:
%sql select distinct year from Movie m, Rating r where m.mID = r.mID AND (r.stars = 5 or r.stars = 4) order by year

Done.


year
1937
1939
1981
2009


Find the titles of all movies that have no ratings.

In [21]:
%sql select title from Movie where mID IN (select mID from Movie except select mID from Rating)

Done.


title
Star Wars
Titanic


Some reviewers didn't provide a date with their rating. Find the names of all reviewers who have ratings with a NULL value for the date. 

In [27]:
%sql select name from Reviewer v, Rating r where v.rID = r.rID and r.ratingDate IS NULL

Done.


name
Daniel Lewis
Chris Jackson


Write a query to return the ratings data in a more readable format: reviewer name, movie title, stars, and ratingDate. Also, sort the data, first by reviewer name, then by movie title, and lastly by number of stars.

In [36]:
%%sql 
select v.name, m.title, r.stars, r.ratingDate from Movie m, Rating r, Reviewer v 
where m.mID = r.mID and r.rID = v.rID
order by v.name, m.title, r.stars

Done.


name,title,stars,ratingDate
Ashley White,E.T.,3,2011-01-02
Brittany Harris,Raiders of the Lost Ark,2,2011-01-30
Brittany Harris,Raiders of the Lost Ark,4,2011-01-12
Brittany Harris,The Sound of Music,2,2011-01-20
Chris Jackson,E.T.,2,2011-01-22
Chris Jackson,Raiders of the Lost Ark,4,None
Chris Jackson,The Sound of Music,3,2011-01-27
Daniel Lewis,Snow White,4,None
Elizabeth Thomas,Avatar,3,2011-01-15
Elizabeth Thomas,Snow White,5,2011-01-19


For all cases where the same reviewer rated the same movie twice and gave it a higher rating the second time, return the reviewer's name and the title of the movie.

In [40]:
%%sql
select A1.name, A1.title from
(select name, title, ratingDate, stars from Reviewer natural join Rating natural join Movie) A1,
(select name, title, ratingDate, stars from Reviewer natural join Rating natural join Movie) A2
where A1.name = A2.name and A1.title = A2.title and A1.ratingDate < A2.ratingDate and A1.stars < A2.stars
group by A1.name
having count(A1.name) = 1

Done.


name,title
Sarah Martinez,Gone with the Wind


For each movie that has at least one rating, find the highest number of stars that movie received. Return the movie title and number of stars. Sort by movie title. 

In [41]:
%%sql
select title, max(stars) from Reviewer natural join Rating natural join Movie
group by title
order by title

Done.


title,max(stars)
Avatar,5
E.T.,3
Gone with the Wind,4
Raiders of the Lost Ark,4
Snow White,5
The Sound of Music,3


For each movie, return the title and the 'rating spread', that is, the difference between highest and lowest ratings given to that movie. Sort by rating spread from highest to lowest, then by movie title. 

In [42]:
%%sql
select title, max(stars) - min(stars) as "rating spread" from Reviewer natural join Rating natural join Movie
group by title
order by "rating spread" desc, title

Done.


title,rating spread
Avatar,2
Gone with the Wind,2
Raiders of the Lost Ark,2
E.T.,1
Snow White,1
The Sound of Music,1


Find the difference between the average rating of movies released before 1980 and the average rating of movies released after 1980. (Make sure to calculate the average rating for each movie, then the average of those averages for movies before 1980 and movies after. Don't just calculate the overall average rating before and after 1980.) 

In [43]:
%%sql
select abs(avg(A1.a) - avg(A2.a)) as answer from
(select title, year, avg(stars) as a from Reviewer natural join Rating natural join Movie
where year >= 1980
group by title) A1,
(select title, year, avg(stars) as a from Reviewer natural join Rating natural join Movie
where year < 1980
group by title) A2

Done.


answer
0.0555555555556


Find the names of all reviewers who rated Gone with the Wind. 

In [51]:
%%sql
select distinct name from Reviewer natural join Rating natural join Movie
where title = "Gone with the Wind"
order by name

Done.


name
Mike Anderson
Sarah Martinez


For any rating where the reviewer is the same as the director of the movie, return the reviewer name, movie title, and number of stars. 

In [52]:
%%sql
select name, title, stars from Reviewer natural join Rating natural join Movie
where name = director

Done.


name,title,stars
James Cameron,Avatar,5


Return all reviewer names and movie names together in a single list, alphabetized.

In [57]:
%%sql
select name as ans from Reviewer
union
select title as ans from Movie
order by ans

Done.


ans
Ashley White
Avatar
Brittany Harris
Chris Jackson
Daniel Lewis
E.T.
Elizabeth Thomas
Gone with the Wind
James Cameron
Mike Anderson


Find the titles of all movies not reviewed by Chris Jackson.

In [58]:
%%sql
select distinct title from Movie
except
select distinct title from Movie natural join Rating natural join Reviewer
where name = 'Chris Jackson'

Done.


title
Avatar
Gone with the Wind
Snow White
Star Wars
Titanic


For all pairs of reviewers such that both reviewers gave a rating to the same movie, return the names of both reviewers. Eliminate duplicates, don't pair reviewers with themselves, and include each pair only once. For each pair, return the names in the pair in alphabetical order. 

In [61]:
%%sql
select distinct A1.name as name_1, A2.name as name_2 from
(select * from Movie natural join Rating natural join Reviewer) A1, 
(select * from Movie natural join Rating natural join Reviewer) A2
where A1.title = A2.title and A1.name < A2.name
order by name_1

Done.


name_1,name_2
Ashley White,Chris Jackson
Brittany Harris,Chris Jackson
Daniel Lewis,Elizabeth Thomas
Elizabeth Thomas,James Cameron
Mike Anderson,Sarah Martinez


For each rating that is the lowest (fewest stars) currently in the database, return the reviewer name, movie title, and number of stars. 

In [63]:
%%sql
select name, title, stars from Movie natural join Rating natural join Reviewer
where stars = (select min(stars) from Movie natural join Rating natural join Reviewer)

Done.


name,title,stars
Sarah Martinez,Gone with the Wind,2
Brittany Harris,The Sound of Music,2
Brittany Harris,Raiders of the Lost Ark,2
Chris Jackson,E.T.,2


List movie titles and average ratings, from highest-rated to lowest-rated. If two or more movies have the same average rating, list them in alphabetical order. 

In [64]:
%%sql 
select title, avg(stars) as a from Movie natural join Rating natural join Reviewer
group by title
order by a desc, title asc

Done.


title,a
Snow White,4.5
Avatar,4.0
Raiders of the Lost Ark,3.33333333333
Gone with the Wind,3.0
E.T.,2.5
The Sound of Music,2.5


Find the names of all reviewers who have contributed three or more ratings.

In [65]:
%%sql
select name from Movie natural join Rating natural join Reviewer
group by name
having count(*) >= 3

Done.


name
Brittany Harris
Chris Jackson


Some directors directed more than one movie. For all such directors, return the titles of all movies directed by them, along with the director name. Sort by director name, then movie title. 

In [66]:
%%sql
select title, director from Movie
where director in
(select director from Movie
group by director
having count(*) > 1)
order by director, title

Done.


title,director
Avatar,James Cameron
Titanic,James Cameron
E.T.,Steven Spielberg
Raiders of the Lost Ark,Steven Spielberg


Find the movie(s) with the highest average rating. Return the movie title(s) and average rating. 

In [68]:
%%sql
select title, a as avg_rating from
(select title, avg(stars) as a from Movie natural join Reviewer natural join Rating
group by title)
where a = (select max(a) from (select avg(stars) as a from Movie natural join Reviewer natural join Rating
group by title))

Done.


title,avg_rating
Snow White,4.5


Find the movie(s) with the lowest average rating. Return the movie title(s) and average rating. 

In [70]:
%%sql
select title, a as avg_rating from
(select title, avg(stars) as a from 
Movie natural join Reviewer natural join Rating 
group by title)
where a = (select min(a) from (select title, avg(stars) as a from 
Movie natural join Reviewer natural join Rating
group by title))

Done.


title,avg_rating
E.T.,2.5
The Sound of Music,2.5


For each director, return the director's name together with the title(s) of the movie(s) they directed that received the highest rating among all of their movies, and the value of that rating. Ignore movies whose director is NULL. 

In [72]:
%%sql
select director, title, max(a) as highest_rating from
(select title, director, max(stars) as a from Movie natural join Reviewer natural join Rating
where director is not null
group by title)
group by director

Done.


director,title,highest_rating
James Cameron,Avatar,5
Robert Wise,The Sound of Music,3
Steven Spielberg,Raiders of the Lost Ark,4
Victor Fleming,Gone with the Wind,4


### Movie rating modifications

Add the reviewer Roger Ebert to your database, with an rID of 209. 

In [74]:
%%sql
insert into Reviewer values (209, 'Roger Ebert')

1 rows affected.


[]

In [75]:
%%sql
select * from Reviewer order by rID, name

Done.


rID,name
201,Sarah Martinez
202,Daniel Lewis
203,Brittany Harris
204,Mike Anderson
205,Chris Jackson
206,Elizabeth Thomas
207,James Cameron
208,Ashley White
209,Roger Ebert


Insert 5-star ratings by James Cameron for all movies in the database. Leave the review date as NULL. 

In [76]:
%%sql
insert into Rating
select 207, mID, 5, null
from Movie

8 rows affected.


[]

In [77]:
%%sql
select * from Rating where stars = 5 order by rID, mID

Done.


rID,mID,stars,ratingDate
206,106,5,2011-01-19
207,101,5,None
207,102,5,None
207,103,5,None
207,104,5,None
207,105,5,None
207,106,5,None
207,107,5,2011-01-20
207,107,5,None
207,108,5,None


For all movies that have an average rating of 4 stars or higher, add 25 to the release year. 

In [78]:
%%sql
update Movie
set year = year + 25 
where mID in
(select mID from Rating
group by mID
having avg(stars) >= 4)

4 rows affected.


[]

In [79]:
%%sql
select * from Movie order by mID

Done.


mID,title,year,director
101,Gone with the Wind,1939,Victor Fleming
102,Star Wars,2002,George Lucas
103,The Sound of Music,1965,Robert Wise
104,E.T.,1982,Steven Spielberg
105,Titanic,2022,James Cameron
106,Snow White,1962,None
107,Avatar,2034,James Cameron
108,Raiders of the Lost Ark,1981,Steven Spielberg


Remove all ratings where the movie's year is before 1970 or after 2000, and the rating is fewer than 4 stars. 

In [80]:
%%sql
delete from Rating
where exists
(select * from
(select * from Rating join Movie using (mID)
where (year < 1970 or year > 2000) and stars < 4) A
where Rating.mID = A.mID and Rating.rID = A.rID and Rating.stars = A.stars and Rating.ratingDate = A.ratingDate)

5 rows affected.


[]

In [81]:
%%sql
select R.rID, R.mID, R.stars, M.title, M.year from Rating R join Movie M on (R.mID = M.mID) order by R.rID, R.mID

Done.


rID,mID,stars,title,year
201,101,4,Gone with the Wind,1939
202,106,4,Snow White,1962
203,108,4,Raiders of the Lost Ark,1981
203,108,2,Raiders of the Lost Ark,1981
205,104,2,E.T.,1982
205,108,4,Raiders of the Lost Ark,1981
206,106,5,Snow White,1962
207,101,5,Gone with the Wind,1939
207,102,5,Star Wars,2002
207,103,5,The Sound of Music,1965


### Connect to social.db and do some queries

In [82]:
%sql sqlite:///social.db

u'Connected: None@social.db'

In [83]:
%sql select * from sqlite_master

Done.


type,name,tbl_name,rootpage,sql
table,Highschooler,Highschooler,2,"CREATE TABLE Highschooler(ID int, name text, grade int)"
table,Friend,Friend,3,"CREATE TABLE Friend(ID1 int, ID2 int)"
table,Likes,Likes,4,"CREATE TABLE Likes(ID1 int, ID2 int)"


In [84]:
%sql select * from Highschooler limit 1

Done.


ID,name,grade
1510,Jordan,9


In [85]:
%sql select * from Friend limit 1

Done.


ID1,ID2
1510,1381


In [86]:
%sql select * from Likes limit 1

Done.


ID1,ID2
1689,1709


Find the names of all students who are friends with someone named Gabriel. 

In [88]:
%%sql
select n as name from
(select ID as i, name as n, grade as g, ID1, ID2 from Highschooler join Friend on ID = ID1) 
join Highschooler on ID2 = Highschooler.ID
where name = 'Gabriel'

Done.


name
Jordan
Cassandra
Andrew
Alexis
Jessica


For every student who likes someone 2 or more grades younger than themselves, return that student's name and grade, and the name and grade of the student they like. 

In [97]:
%%sql
select n, g, name, grade from
(select ID as i, name as n, grade as g, ID1, ID2 from Highschooler join Likes on ID = ID1) 
join Highschooler on ID2 = Highschooler.ID
where g - grade >= 2

Done.


n,g,name,grade
John,12,Haley,10


For every pair of students who both like each other, return the name and grade of both students. Include each pair only once, with the two names in alphabetical order. 

In [98]:
%%sql
select * from
(select n, g, name, grade from
(select ID as i, name as n, grade as g, ID1, ID2 from Highschooler join Likes on ID = ID1) join Highschooler on ID2 = Highschooler.ID
intersect
select name, grade, n, g from
(select ID as i, name as n, grade as g, ID1, ID2 from Highschooler join Likes on ID = ID1) join Highschooler on ID2 = Highschooler.ID)
where n < name

Done.


n,g,name,grade
Cassandra,9,Gabriel,9
Jessica,11,Kyle,12


Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their names and grades. Sort by grade, then by name within each grade. 

In [99]:
%%sql
select name, grade from Highschooler where ID not in
(select ID1 from Likes
union
select ID2 from Likes)
order by grade, name

Done.


name,grade
Jordan,9
Tiffany,9
Logan,12


For every situation where student A likes student B, but we have no information about whom B likes (that is, B does not appear as an ID1 in the Likes table), return A and B's names and grades. 

In [100]:
%%sql
select n, g, name, grade from
(select ID as i, name as n, grade as g, ID1, ID2 from Highschooler join Likes on ID = ID1) join Highschooler on ID2 = Highschooler.ID
where ID2 not in (select ID1 from Likes)

Done.


n,g,name,grade
Alexis,11,Kris,10
Brittany,10,Kris,10
Austin,11,Jordan,12
John,12,Haley,10


Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, then by name within each grade. 

In [101]:
%%sql
select n, g from
(select ID as i, name as n, grade as g, ID1, ID2 from Highschooler join Friend on ID = ID1) join Highschooler on ID2 = Highschooler.ID
group by i
having max(grade) = min(grade)
order by g, n

Done.


n,g
Jordan,9
Brittany,10
Haley,10
Kris,10
Gabriel,11
John,12
Logan,12


For each student A who likes a student B where the two are not friends, find if they have a friend C in common (who can introduce them!). For all such trios, return the name and grade of A, B, and C.

In [102]:
%%sql
select H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade 
from Highschooler H1 join Highschooler H2 join Highschooler H3 join
(select A.ID1 as a, A.ID2 as b, F1.ID2 as c 
from (select * from Likes except select * from Friend) A join Friend F1 join Friend F2 
on A.ID1 = F1.ID1 and A.ID2 = F2.ID1
where F1.ID2 = F2.ID2) L 
on H1.ID = L.a and H2.ID = L.b and H3.ID = L.c

Done.


name,grade,name_1,grade_1,name_2,grade_2
Austin,11,Jordan,12,Kyle,12
Andrew,10,Cassandra,9,Gabriel,9
Austin,11,Jordan,12,Andrew,10


Find the difference between the number of students in the school and the number of different first names. 

In [103]:
%%sql
select c - n from
(select count(ID) as c from Highschooler),
(select count(distinct name) as n from Highschooler)

Done.


c - n
2


Find the name and grade of all students who are liked by more than one other student. 

In [104]:
%%sql
select name, grade from Likes L join Highschooler H on L.ID2 = H.ID
group by ID
having count(*) > 1

Done.


name,grade
Kris,10
Cassandra,9


For every situation where student A likes student B, but student B likes a different student C, return the names and grades of A, B, and C. 

In [105]:
%%sql
select H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade 
from Highschooler H1 join Likes L1 join Highschooler H2 join Likes L2 join Highschooler H3 
on H1.ID = L1.ID1 and L1.ID2 = H2.ID and H2.ID = L2.ID1 and L2.ID2 = H3.ID
where H3.ID <> H1.ID

Done.


name,grade,name_1,grade_1,name_2,grade_2
Andrew,10,Cassandra,9,Gabriel,9
Gabriel,11,Alexis,11,Kris,10


Find those students for whom all of their friends are in different grades from themselves. Return the students' names and grades.

In [106]:
%%sql
select H1.name, H1.grade from Highschooler H1 join Friend F join Highschooler H2 
on H1.ID = F.ID1 and F.ID2 = H2.ID
except
select distinct H1.name, H1.grade from Highschooler H1 join Friend F join Highschooler H2 
on H1.ID = F.ID1 and F.ID2 = H2.ID
where H1.grade = H2.grade

Done.


name,grade
Austin,11


What is the average number of friends per student?

In [107]:
%%sql
select cast(f as float) / cast(s as float)
from (select count(*) as s from Highschooler),
(select count(*) as f from Friend)

Done.


cast(f as float) / cast(s as float)
2.5


Find the number of students who are either friends with Cassandra or are friends of friends of Cassandra. Do not count Cassandra, even though technically she is a friend of a friend. 

In [108]:
%%sql
select count(*) from
(select F1.ID2 from Highschooler H1 join Friend F1 join Friend F2 join Highschooler H2 
on H1.ID = F1.ID1 and F1.ID2 = F2.ID1 and F2.ID2 = H2.ID
where H1.name = 'Cassandra' and H2.name is not 'Cassandra'
union
select F2.ID2 from Highschooler H1 join Friend F1 join Friend F2 join Highschooler H2 
on H1.ID = F1.ID1 and F1.ID2 = F2.ID1 and F2.ID2 = H2.ID
where H1.name = 'Cassandra' and H2.name is not 'Cassandra')

Done.


count(*)
7


Find the name and grade of the student(s) with the greatest number of friends. 

In [109]:
%%sql
select name, grade from Highschooler H join Friend F on H.ID = F.ID1
group by ID
having count(*) =
(select max(c) from
(select count(*) as c from Highschooler H join Friend F on H.ID = F.ID1
group by ID))

Done.


name,grade
Alexis,11
Andrew,10


### social.db modifications

Remove all 12th graders from Highschooler. 

In [110]:
%%sql
delete from Highschooler
where grade = 12

4 rows affected.


[]

In [111]:
%%sql
select * from Highschooler order by ID

Done.


ID,name,grade
1101,Haley,10
1247,Alexis,11
1316,Austin,11
1381,Tiffany,9
1468,Kris,10
1501,Jessica,11
1510,Jordan,9
1641,Brittany,10
1689,Gabriel,9
1709,Cassandra,9


If two students A and B are friends, and A likes B but not vice-versa, remove the Likes tuple. 

In [112]:
%%sql
delete from Likes
where exists
(select * from
	(select * from Likes intersect select * from Friend
	except
	select L1.ID1, L1.ID2 from Likes L1 join Likes L2 on L1.ID2 = L2.ID1 where L1.ID1 = L2.ID2) A
where A.ID1 = Likes.ID1 and A.ID2 = Likes.ID2)

2 rows affected.


[]

In [113]:
%%sql
select H1.name, H1.grade, H2.name, H2.grade 
from Likes L, Highschooler H1, Highschooler H2 
where L.ID1 = H1.ID and L.ID2 = H2.ID 
order by H1.name, H1.grade

Done.


name,grade,name_1,grade_1
Alexis,11,Kris,10
Andrew,10,Cassandra,9
Cassandra,9,Gabriel,9
Gabriel,9,Cassandra,9


For all cases where A is friends with B, and B is friends with C, add a new friendship for the pair A and C. Do not add duplicate friendships, friendships that already exist, or friendships with oneself.

In [114]:
%%sql
insert into Friend
select distinct F1.ID1, F2.ID2 from Friend F1 join Friend F2 on F1.ID2 = F2.ID1 where F1.ID1 <> F2.ID2
except
select * from Friend

54 rows affected.


[]

In [115]:
%%sql
select ID, name, grade, (select count(*) from Friend where id1 = H.id) from Highschooler H order by ID

Done.


ID,name,grade,id)
1101,Haley,10,3
1247,Alexis,11,7
1316,Austin,11,6
1381,Tiffany,9,6
1468,Kris,10,6
1501,Jessica,11,7
1510,Jordan,9,5
1641,Brittany,10,3
1689,Gabriel,9,8
1709,Cassandra,9,7
